In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
file_name='2020年12月31日周四'

with open ('./{}.txt'.format(file_name)) as f:
    text=f.read()
    f.close()


In [21]:
fenlei={'短融':[],
        '中票':[],
        '企业债':[],
        '其他':[]}

need_flag=True
for i,line in enumerate(text.split('\n')):
    line=re.split('[\t ]+',line.strip())
    if len(line)==1 and line!=['']:
        if re.search('|'.join(['短融','中票','企业债','其他']) ,line[0])!=None:
            now_list=re.search('|'.join(['短融','中票','企业债','其他']) ,line[0]).group()
            need_flag=True
        else:
            need_flag=False
    if not need_flag:
        continue # 如果为非所需的数据，跳过
    if len (line)>=5:
        """
        
        
        
        """
        
        year=line[0]
        try:
#             rating=line[['AAA|(A\-1)' in i for i in line ].index(True)] # 找评级
            rating=line[[re.search('AAA|(A\-1)',i)!=None  for i in line].index(True)] #找评级
            ytm=line[[re.search('\d+\.\d+|\d+',i)!=None  for i in line[3:]].index(True)+3].replace('%','') #找ytm
            name=line[[re.search('[^\x00-\xff]+',i)!=None  for i in line[1:]].index(True)+1]#找名称
            fenlei[now_list].append([year,name,ytm,rating])
            print(line,ytm)
            
        except:
            pass
#             if now_list=='短融':
#                 print('无法获取信息',line)
            
        
    else:
        pass
        

len(fenlei['短融'])

['207D', '20南通沿海SCP002', '012003753', 'AAA', '3.82'] 3.82
['0.93Y+2Y', '18长电MTN001', '101801418', 'AAA', '3.10', '行权'] 3.10
['1.47Y', '12中船MTN1', '1282205', 'AAA', '3.30'] 3.30
['1.68Y+N', '19东航MTN001', '101901211', 'AAA', '3.79'] 3.79
['2.35Y', '20正泰MTN001', '102000965', 'AA+/AAA', '4.20'] 4.20
['2.65Y+N', '20京能电力MTN001', '102001636', 'AAA', '4.25'] 4.25
['2.78Y', '18冀港集MTN002', '101801144', 'AAA', '3.99'] 3.99
['2.81Y+N', '20华侨城MTN005', '102001955', 'AAA', '4.25'] 4.25
['2.81Y+N', '20华侨城MTN005', '102001955', 'AAA', '4.25'] 4.25
['2.84Y+N', '20华侨城MTN006', '102002077', 'AAA', '4.26'] 4.26
['0.82Y+2Y', '16油服04', '136767', 'AAA', '3.29', '行权'] 3.29
['1.14Y', '19长电01', '155177', 'AAA', '3.10'] 3.10
['1.14Y', '19长电01', '155177', 'AAA', '3.10'] 3.10
['1.22Y+2Y', '19国新01', '155274', 'AAA', '3.20', '行权'] 3.20
['1.44Y', '19建材07', '155342', 'AAA', '3.25'] 3.25
['2.93Y+6Y', '20青城05', '149320', 'AAA', '3.69', '行权'] 3.69
['4.28Y', '20沪国01', '163387', 'AAA', '3.84'] 3.84
['4.98Y', '15正定棚改项目债', '152

2

In [17]:
fenlei_df ={i:pd.DataFrame(fenlei[i],columns='剩余期限 简称 收益率 评级'.split(' ')) for  i in fenlei.keys()}
fenlei_df['其他']

,剩余期限,简称,收益率,评级
0,348D,18厦门国际银行,3.18,AAA
1,1.55Y,19西南02,3.28,AAA
2,1.55Y,19西南02,3.28,AAA
3,1.94Y,20华泰G8,3.30,AA/AAA
4,1.94Y,20华泰G8,3.30,AA/AAA
5,2.24Y,20信达01,3.58,AAA
6,2.24Y,20信达01,3.58,AAA
7,2.24Y,20信达01,3.58,AAA
8,2.29Y,20华夏银行绿色01,3.39,AAA
9,2.53Y,20中证13,3.40,AA/AAA


In [18]:
def get_day(days:str):
    main_day=days.split('+')[0]
    if 'D' in main_day:
        real_day=float(re.findall('\d+\.\d+|\d+',main_day)[0])
    elif 'Y' in main_day:
        real_day=float(re.findall('\d+\.\d+|\d+',main_day)[0])*365
    else:
        real_day=main_day
    return real_day

In [19]:
for key in fenlei_df:
    df=fenlei_df[key]
    df['day']=df['剩余期限'].apply(lambda x:get_day(x))
    df.sort_values('day',inplace=True)
    df.index=[i for i in range(len(df))]
    fenlei_df[key]=df.drop('day',axis=1) #去掉辅助列
    
fenlei_df['中票'].head()

,剩余期限,简称,收益率,评级
0,295D,18铜陵有色MTN003,4,AAA
1,306D,18鞍钢MTN001,3.95,AAA
2,0.93Y+2Y,18长电MTN001,3.10,AAA
3,353D(休1),18中银投资MTN001,3.25,AAA
4,1.06Y,19中电信MTN001,3.11,AAA


In [7]:
fenlei_df['其他']

,剩余期限,简称,收益率,评级
0,348D,18厦门国际银行,3.18,AAA
1,1.55Y,19西南02,3.28,AAA
2,1.55Y,19西南02,3.28,AAA
3,1.94Y,20华泰G8,3.30,AA/AAA
4,1.94Y,20华泰G8,3.30,AA/AAA
5,2.24Y,20信达01,3.58,AAA
6,2.24Y,20信达01,3.58,AAA
7,2.24Y,20信达01,3.58,AAA
8,2.29Y,20华夏银行绿色01,3.39,AAA
9,2.53Y,20中证13,3.40,AA/AAA


In [8]:
pd.concat([df for df in fenlei_df.values()])

,剩余期限,简称,收益率,评级
0,207D,20南通沿海SCP002,3.82,AAA
1,250D,20陕投集团SCP002,3.44,AAA
0,295D,18铜陵有色MTN003,4,AAA
1,306D,18鞍钢MTN001,3.95,AAA
2,0.93Y+2Y,18长电MTN001,3.10,AAA
...,...,...,...,...
18,2.87Y,20交通银行02,3.42,AAA
19,2.87Y,20交通银行02,3.42,AAA
20,2.96Y,20华夏银行,3.41,AAA
21,2.96Y,20华夏银行,3.41,AAA


## 输出到excel中

In [118]:
from shutil import copyfile
copyfile('./页面模板.xlsx','./{}.xlsx'.format(file_name))

import openpyxl
wb=openpyxl.load_workbook('{}.xlsx'.format(file_name))
sht=wb['Sheet1']
sht



<Worksheet "Sheet1">

In [119]:

def get_excel_coor(now_line):
    page=int(now_line/120)
    line=now_line%60
    if now_line%120<60:
        col='A'
    else:
        col='E'
    row=page*62+line+3
    return str(col)+str(row)
now_line=120
print(get_excel_coor(now_line))


A65


In [120]:
from openpyxl.styles import Font, colors, Alignment
def small_title(sheet,coor,title):
    end_coor_dic={'A':'D','E':'H'}
    hebing=coor+':'+end_coor_dic[coor[0]]+coor[1:]
    sheet.merge_cells(hebing)
    sheet[coor]=title
    sheet[coor].alignment = Alignment(horizontal='center', vertical='center')
    print(hebing,title)
small_title(sht,'A3','短融')

A3:D3 短融


In [121]:
# 取消合并所有单元格

for page in range(6):
    page_range='A{}:H{}'.format(page*62+3,page*62+62)
    try:
        sht.unmerge_cells(page_range)
    except:
        print('unmerged')
        pass
    # Borders(11) 内部垂直边线。    # Borders(12) 内部水平边线。

#     for line in [11,12]:
#         sht.c(page_range).api.Borders(line).LineStyle = 1
#         sht.range(page_range).api.Borders(line).Weight = 2
#         sht.range(page_range).api.HorizontalAlignment = -4131
#         sht.range(page_range).value=None
#         sht.range(page_range).api.NumberFormat = "@"
sht['E1']=file_name


unmerged
unmerged
unmerged
unmerged
unmerged
unmerged


In [128]:
def write_area(sht,coor,write_df):
    if coor[0]=='A':
        min_col=1
        max_col=4
    if coor[0]=='E':
        min_col=5
        max_col=8
    min_row=int(coor[1:])
    max_row=min_row+len(write_df)-1
#     print(write_df)
    for i,row in enumerate(sht.iter_rows(min_row=min_row,min_col=min_col, max_col=max_col, max_row=max_row)):
        for j,cell in enumerate(row):
#             print(cell)
            try:
                cell.value=write_df.iloc[i,j]
            except:
                print(cell)

In [129]:
now_line=0


def xiugai_df(write_df):
    buf=write_df.iloc[0,0]
    write_df.set_index('剩余期限',inplace=True)
    write_df.columns=write_df.iloc[0]
    write_df.index.name=buf
    write_df=write_df.iloc[1:] # 通过一个很愚蠢的方式去掉表头
    return write_df

for one in fenlei_df:
    # 输入标签
    coor=get_excel_coor(now_line)
    small_title(sht,coor,one)
    now_line+=1
    
    
    if len(fenlei_df[one])==0:
        continue
    
    line_left=60-now_line%60# 剩下多少行的空间
    small_df=fenlei_df[one]
    
    while len(small_df)>0:
#         print('还剩多少行:',line_left,'需要多少行:',len(small_df))
        coor=get_excel_coor(now_line) # 计算新的坐标位置
        print(coor)
        # 填充剩余位置
        write_df=small_df.iloc[:line_left]
        
        write_area(sht,coor,write_df)
        
        # 更新位置
        now_line+=len(write_df)
        
        small_df=small_df.iloc[line_left:]
        line_left=60-now_line%60  # 更新剩下多少行的空间
        


A3:D3 短融
A4:D4 中票
A5
E3
A65
E65
E78:H78 企业债
E79
A127
E127
A189
A247:D247 其他
A248
E189
A251


In [131]:
wb.save('{}.xlsx'.format(file_name))